In [4]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import t


In [3]:
# Step 1: Organize the data
data = pd.DataFrame({
    "fatigue_resistance": [
        355, 395, 323, 356, 393, 327,  # Group A, F-15
        415, 403, 446, 410, 409, 440,  # Group A, F-16
        392, 350, 348, 389, 353, 350,  # Group B, F-15
        396, 375, 357, 390, 380, 360   # Group B, F-16
    ],
    "production_method": [
        "A", "A", "A", "A", "A", "A",  # Group A
        "A", "A", "A", "A", "A", "A",  # Group A
        "B", "B", "B", "B", "B", "B",  # Group B
        "B", "B", "B", "B", "B", "B"   # Group B
    ],
    "aircraft_type": [
        "F-15", "F-15", "F-15", "F-15", "F-15", "F-15",  # F-15
        "F-16", "F-16", "F-16", "F-16", "F-16", "F-16",  # F-16
        "F-15", "F-15", "F-15", "F-15", "F-15", "F-15",  # F-15
        "F-16", "F-16", "F-16", "F-16", "F-16", "F-16"   # F-16
    ]
})

# Step 2: Specify the Two-Way ANOVA model with interaction
formula = "fatigue_resistance ~ production_method + aircraft_type + production_method:aircraft_type"
model = ols(formula, data=data).fit()

# Step 3: Perform the ANOVA
table = sm.stats.anova_lm(model, typ=2)  # Type II sums of squares
print(table)

                                      sum_sq    df          F    PR(>F)
production_method                2242.666667   1.0   4.572671  0.045012
aircraft_type                    8437.500000   1.0  17.203589  0.000498
production_method:aircraft_type  3700.166667   1.0   7.544432  0.012436
Residual                         9809.000000  20.0        NaN       NaN


In [5]:

# Step 4: Construct 95% confidence intervals for group means
means = data.groupby(["production_method", "aircraft_type"]).mean()["fatigue_resistance"]
stderr = data.groupby(["production_method", "aircraft_type"]).sem()["fatigue_resistance"]

df_residual = model.df_resid  # Residual degrees of freedom
alpha = 0.05
critical_t = t.ppf(1 - alpha / 2, df_residual)  # Two-tailed critical t-value

conf_intervals = {}
for group in means.index:
    mean = means[group]
    se = stderr[group]
    margin_of_error = critical_t * se
    conf_intervals[group] = (mean - margin_of_error, mean + margin_of_error)

print("95% Confidence Intervals:")
for group, ci in conf_intervals.items():
    print(f"{group}: {ci}")


95% Confidence Intervals:
('A', 'F-15'): (331.8007891579079, 384.53254417542547)
('A', 'F-16'): (405.22111869430194, 435.77888130569806)
('B', 'F-15'): (345.8958033212458, 381.43753001208756)
('B', 'F-16'): (362.98027041367243, 389.6863962529942)


In [6]:
# Repeat for aircraft_type if needed
formula_one_way_aircraft = "fatigue_resistance ~ aircraft_type"
model_one_way_aircraft = ols(formula_one_way_aircraft, data=data).fit()
one_way_table_aircraft = sm.stats.anova_lm(model_one_way_aircraft, typ=2)

print("\nOne-Way ANOVA Results for Aircraft Type:")
print(one_way_table_aircraft)



One-Way ANOVA Results for Aircraft Type:
                     sum_sq    df          F    PR(>F)
aircraft_type   8437.500000   1.0  11.784343  0.002377
Residual       15751.833333  22.0        NaN       NaN
